# Lab 4: Ice Stream Margin, continued
---

## Submission

Please upload your completed notebook (renamed with your name) to Canvas as usual.

## 3. The Inverse Problem

Recall last week's lab, where you wrote some functions to calculate $\dot{u}_1$ and $A$. In part 2.3, you calculated an $A$ value from each of the three data points from `isbdata.mat` and then used those, along with setting $n = 3$, to plot synthetic (forward-modeled) velocity profiles to compare with the observed velocity profile. Today we'll explore a little more forward modeling before defining and solving the inverse problem. You'll want to have your previous lab notebook handy since you'll be using equations and code defined during that lab.

First, let's load in the data using the same code as last week, copied for your convenience:

In [1]:
from scipy.io import loadmat

!curl -O https://raw.githubusercontent.com/uafgeoteach/GEOS631_FoG/master/labs/lab_03/isbdata.mat
    
data = loadmat('isbdata.mat', squeeze_me=True)

x3 = data['dpx'] * 1000  # [m]
u1 = data['dpv']  # [m/yr]

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   854  100   854    0     0   4229      0 --:--:-- --:--:-- --:--:--  4248


### 3.1 Vary $n$

Now create a `for` loop to calculate the predicted velocity profile — for one estimated $A$ (from a single data point) — for each $n$. Plot all of these on top of the data in different colors, and add a legend. You can loop over the values of $n$ using:

```python
for n in range(1, 8):
    # Your plotting code goes here
```

Just using these approximated $A$ values, do you think $n = 3$ is the best solution? How different are the $A$ values for each $n$? What are the units for each $A$? Include your code in a **new code cell below** and discussion in a **new text cell below**.

### 3.2 Vary $A$ again, systematically

Go back to $n=3$ (you can modify your `for` statement above to `for n in [3]:` as a shortcut). So far you have only used a single estimated $A$ value. Now add another `for` loop inside the one you just made in order to try some different values of $A$. Base your range of $A$ values on the variations you saw for $n = 3$ in Section 2.3 of last lab. Plot all of your profiles for different $A$ values for $n = 3$ on a single plot, along with the data. Add a legend as well. You can use the [`np.logspace()`](https://numpy.org/doc/stable/reference/generated/numpy.logspace.html) command to generate evenly distributed values for $A$ around your estimated $A$ value. Include your code in a **new code cell below** (note that most of it can be copied from your previous answer).

### 3.3 Grid Calculations of Velocity

You can now extend the calculations to all values of $n$. You need to think about what information you want to save outside of these looped calculations. For each $n$ and $A$, you are calculating an array that is your velocity profile. You can store the velocity profile for each of these in a 3D array which you can pre-allocate like

```python
u1_pred_all = np.empty((n_vec.size, A_vec.size, u1.size))
```

where `n_vec` and `A_vec` are arrays of $n$ values and $A$ values, respectively. Then you can loop over the $n$ values with `i` and the $A$ values with `j` and calculate a profile `u1_pred` for each, which can be assigned to its place within the loop using:

```python
u1_pred_all[i, j, :] = u1_pred
```

Do this in a **new code cell below**.

### 3.4 Defining the cost function

Now that you have calculated many profiles with different $n$ and $A$ values, you need to decide which one is the *best* one. To do this we define a "cost" function. A cost function, also called a mismatch function, is one that has a low value when the fit of your model predictions to the data is good and a high value when the fit is bad. You have a number of options for defining your cost function. For example:

* The sum of the absolute value of the differences between each of your predicted velocities and each data point (the $L_1$ norm)

$$\left\Vert \dot{u}^\mathrm{obs}-\dot{u}^\mathrm{pred} \right\Vert_1 = \sum_j \left|\dot{u}_j^\mathrm{obs}-\dot{u}_j^\mathrm{pred}\right|$$

* The square root of the sum of the squares of the differences between each of your predicted velocities and each data point (the $L_2$ norm)

$$\left\Vert \dot{u}^\mathrm{obs}-\dot{u}^\mathrm{pred} \right\Vert_2 = \sqrt{\sum_j \left(\dot{u}_j^\mathrm{obs}-\dot{u}_j^\mathrm{pred}\right)^2}$$

There are others you might use for other models, such as wanting the smoothest curve (the second derivative to be small). Ultimately, your problem dictates which cost-function is appropriate. The generic choice is $L_2$, however, large outliers in your data can bias the $L_2$ solution (as the square amplifies their error). In such cases, $L_1$ is a more appropriate choice. 

For this study, we also want to be able to allow for the level of error (or confidence) in the measurement to influence the outcome. Data with large uncertainties should have a lower impact on the fit than data that we are very confident in. Echelmeyer states the errors in his measurements to be about $\delta=3$–$5$ m/yr. An example cost function that uses the $L_1$ norm and incorporates the errors is:

$$ J_1=\frac{1}{N-p} \sum^N_j \frac{\left|{\dot{u}_j^\mathrm{obs}}-{\dot{u}_j^\mathrm{pred}}\right|}{\delta_j} $$

Where $N$ is the number of observations and $p$ is the number of unknowns, such that $N-p$ is the number of degrees of freedom. In this equation $\delta$ does not need to be the same for every observation. Note that when $\delta_j$ is large (i.e., the error is large) the corresponding datapoint is down-weighted. The $L_2$ norm version of this is:

$$ J_2=\frac{1}{N-p} \sqrt{\sum^N_j \frac{\left({\dot{u}_j^\mathrm{obs}}-{\dot{u}_j^\mathrm{pred}}\right)^2}{\delta_j^2}} $$

Finally, you may want to downweight some of the observations. Note, for example, that the observations far in the margin curve off of the model predicted values — the model only predicts right to the margin edge. You might need to use a weighting function to avoid trying to fit a solution to these data points. To do this, you can insert a weighting vector $w_j$ into the norm calculation. Here's how this looks for the $L_2$ norm version, for example:

$$ J_{2,\mathrm{w}}=\frac{1}{N-p} \sqrt{\sum^N_j w_j \frac{\left({\dot{u}_j^\mathrm{obs}}-{\dot{u}_j^\mathrm{pred}}\right)^2}{\delta_j^2}} $$

You can construct $w_j$ via e.g.

```python
w = np.ones(u1.size)  # Make an appropriately sized vector of ones
w[-2:] = 0  # Make the last few entries (two, in this case) zero
```

Look at the data above and assess which cost function is appropriate here. In a **new code cell below**, add its calculation (either $J_1$ or $J_2$, weighted or unweighted) into your nested for loops. Also, please **include in a comment** your justification for selection of $J_1$ or $J_2$ along with weighting or not.

### 3.5 Visualizing the misfit space

At this point, your code should give you the values of $J$ for a range of $n$ and $A$ values. The best solution (values for $n$ and $A$) will be one that minimize a cost function $J$. 

There are several ways to visualize a misfit space. Feel free to explore options using [`ax.contourf()`](https://matplotlib.org/3.3.1/api/_as_gen/matplotlib.axes.Axes.contourf.html#matplotlib.axes.Axes.contourf) or other 2D visualization tools. At minimum, however, do the following for your cost function: Plot the $J$ as a function of $A$ for each $n$ — i.e., a separate curve for each $n$.

> **Hints:** You will want to log-space your $A$ axis. You'll also want to plot the log of $J$. You can use `np.log10()` for this.

### 3.6 Finding the best solution

Use `np.min()` to find the minimum value of $J$, then figure out which $n$ and $A$ correspond to this best solution. Plot the velocity profile for your best solution on the same plot with the original data.

This solution process is called a "grid search," because we simply calculated many forward models for a subset of (we think) a reasonable parameter space and then we are choosing the best one to call our best solution to the inverse problem. This is a very simplistic way to approach an inverse problem, but not necessarily the most efficient, or one that guarantees the best / most reasonable solution. For instance, if your grid is too big (too many parameter combinations to test), then you can imagine that we run into computational limits. If you choose the parameters ranges that you test poorly, you may not include the parameter space for the global minimum of the misfit. There are other techniques that more efficiently and completely probe the parameter space to minimize a cost function. If you want to learn more about inverse problems, take Carl Tape's class!

## 4. Final words

Your best solution is unlikely to fit perfectly (as your eye can see) compared to the data in terms of the general shape. How close to the $n = 3$ value for glacier ice is the $n$ value that best fits? Consider the assumptions you made in calculating the stress tensor and the equation for velocity. In a **new text cell below**, choose two of your assumptions that you think might be not strong enough and explain how you think altering those assumptions might lead to a subtly different solution.